<a href="https://colab.research.google.com/github/Paporisarma09/taxi-fare-prediction/blob/main/22f1001482_notebook_t32023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'taxi-fare-guru-total-amount-prediction-challenge:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F61246%2F6604167%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240609%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240609T154141Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0a04a6f5f7c3c63b79d3712fcea74c643707163169a64227c34fef5231c2d212ba6264edb8486c5a9bf85fd9a03a41c27051107b97956e90e1d0348a2e194c46d1ea54e3bfff4989a50099a4e58379bf534e884a6ce15a5b4bb6442a5d80d17a8e82289682da1a4c10b835268f9228d187b5636bf2d84a05f8eec4cc3cf6c2918c39d239c81252bb811bbb5bb28dde0254189d8f3864ea24e2f1fd9f79c9e3ae61e30efaa23257d1996515c4172e5bbab33b16ead44f735e3a5958eec85aba25c39acd5ba068fd4ce7772da4d2cca4667236d9460e6896bdfddadf99e7c7c0ca58e0cf2f9b0bec4a7b832cfe553cbd3cb6b2ecf23f2a53c6576553fa2628469a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample=pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/sample.csv.csv')
sample.head()

# Milestone1

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,cross_validate
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,OrdinalEncoder,MaxAbsScaler,PolynomialFeatures,StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, SGDRegressor,Ridge,Lasso,RidgeCV,LassoCV,ElasticNetCV
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,median_absolute_error
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,AdaBoostRegressor
from sklearn.svm import SVR,LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA


In [ ]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits= 10, test_size=0.2, random_state=42)

# Loading the Datasets

In [ ]:
train_data= pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv')
test_data=pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv')

# EDA
# Insights from EDA:
1. train data shape is (175000, 17) and test data shape is(50000, 16).
2. there are 13 numerical features and 4 are categorical features in train data.
3. there are 12 numerical features and 4 are categorical features in train data.
4. 5 features are having null vlues out of them 2 are categorical and 3 of them numerical in train and test data.
5. there is no duplicate values in train and test data.
6. In the categorical feature 'store_and_fwd_flag',there are three types of values. Those are 'Y' means yes, 'N' means no and 'NAN'. Out of all three 'N' frequencies are more.
7. In the categorical feature 'payment_type',there are 5 types of values. Those are 'Credit Card', 'Cash', 'Wallet', 'UPI', 'unknown'. Out of all 5, 'Credit Card' frequencies are more.
8. 'passenger_count', 'congestion_surcharge', 'Airport_fee' are having missing values.

# Checking the heads of each dataset to get rough idea about them

In [ ]:
train_data.head()

In [ ]:
test_data.head()

# Checking the shape of each dataset

In [ ]:
train_data.shape

In [ ]:
test_data.shape

# Checking the info of each dataset for getting the no of rows,null values,datatype of each columns

In [ ]:
train_data.info()

In [ ]:
test_data.info()

 # Checking the null values in every column of each dataset which we need to handle

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

# Checking Duplicate values in Data

In [ ]:
train_data.duplicated().sum()

In [ ]:
test_data.duplicated().sum()

# Getting the idea about the data and their distributions through histogram

In [ ]:
train_data.hist(edgecolor = 'blue',bins = 60, figsize=(30,20))
plt.subplots_adjust(hspace = 0.5,wspace = 0.3)
plt.show()

In [ ]:
train_data['store_and_fwd_flag'].unique()

In [ ]:
print(len(train_data[train_data['store_and_fwd_flag'] == 'Y']))
print(len(train_data[train_data['store_and_fwd_flag'] == 'N']))
print(len(train_data[train_data['store_and_fwd_flag'].isna()]))

In [ ]:
train_data['payment_type'].unique()

In [ ]:
print(len(train_data[train_data['payment_type'] == 'Credit Card']))
print(len(train_data[train_data['payment_type'] == 'Cash']))
print(len(train_data[train_data['payment_type'] == 'Wallet']))
print(len(train_data[train_data['payment_type'] == 'UPI']))
print(len(train_data[train_data['payment_type'] == 'unknown']))


In [ ]:
sns.catplot(data=train_data,x='payment_type',kind='count')
plt.show()

In [ ]:
sns.catplot(data=train_data,x='store_and_fwd_flag',kind='count')
plt.show()

# Split the dataset in training and validation sets with train_test_split

In [ ]:
X = train_data.drop(columns=['total_amount'])
y=train_data['total_amount']

In [ ]:
X_train, X_val,y_train, y_val = train_test_split(X, y, test_size=0.2,random_state=42)

# Feature Engeneering

In [ ]:
X_train['travel_time'] = ((pd.to_datetime(X_train['tpep_dropoff_datetime'])- pd.to_datetime(X_train['tpep_pickup_datetime'])).dt.seconds)/60
X_train = X_train.drop(columns = ['tpep_dropoff_datetime','tpep_pickup_datetime'])

In [ ]:
X_val['travel_time'] = ((pd.to_datetime(X_val['tpep_dropoff_datetime'])- pd.to_datetime(X_val['tpep_pickup_datetime'])).dt.seconds)/60
X_val = X_val.drop(columns = ['tpep_dropoff_datetime','tpep_pickup_datetime'])

In [ ]:
Num_cols = X_train.select_dtypes(include = 'number').columns.to_list()
print(Num_cols)

In [ ]:
Cat_cols = X_train.select_dtypes(exclude = 'number').columns.to_list()
print(Cat_cols)

# Handling missing values in the columns

In [ ]:
X_train['store_and_fwd_flag'].unique()

In [ ]:
print(len(X_train[X_train['store_and_fwd_flag'] == 'Y']))
print(len(X_train[X_train['store_and_fwd_flag'] == 'N']))
print(len(X_train[X_train['store_and_fwd_flag'].isna()]))

In [ ]:
X_val['store_and_fwd_flag'].unique()

In [ ]:
print(len(X_val[X_val['store_and_fwd_flag'] == 'Y']))
print(len(X_val[X_val['store_and_fwd_flag'] == 'N']))
print(len(X_val[X_val['store_and_fwd_flag'].isna()]))

In [ ]:
X_train['store_and_fwd_flag'].fillna('N',inplace=True)
print(X_train['store_and_fwd_flag'].isna().sum())

In [ ]:
X_val['store_and_fwd_flag'].fillna('N',inplace=True)
print(X_val['store_and_fwd_flag'].isna().sum())

In [ ]:
print(X_train['payment_type'].unique())

In [ ]:
print(len(X_train[X_train['payment_type'] == 'Credit Card']))
print(len(X_train[X_train['payment_type'] == 'Cash']))
print(len(X_train[X_train['payment_type'] == 'Wallet']))
print(len(X_train[X_train['payment_type'] == 'UPI']))
print(len(X_train[X_train['payment_type'] == 'unknown']))

In [ ]:
X_train['payment_type'].replace('unknown', 'Credit Card', inplace = True)
X_val['payment_type'].replace('unknown', 'Credit Card', inplace = True)

In [ ]:
X_train['payment_type'].unique()

In [ ]:
X_val['payment_type'].unique()

In [ ]:
X_train['store_and_fwd_flag'].unique()

In [ ]:
X_val['store_and_fwd_flag'].unique()

In [ ]:
# Using simple imputer to fill the null values in numerical columns
si= SimpleImputer(strategy= 'mean')
X_train[Num_cols]= si.fit_transform(X_train[Num_cols])
X_val[Num_cols]= si.transform(X_val[Num_cols])

In [ ]:
X_train.isnull().sum()

In [ ]:
X_val.isnull().sum()

In [ ]:
X_train.dtypes

In [ ]:
# One-Hot Encoding using get_dummies in categorical columns
X_train = pd.get_dummies(X_train, columns=Cat_cols, prefix=['store_and_fwd_flag ','payment_type'],dtype=bool)
X_train=X_train*1
X_val = pd.get_dummies(X_val, columns=Cat_cols, prefix=['store_and_fwd_flag ','payment_type'],dtype=bool)
X_val=X_val*1

In [ ]:
X_train.dtypes

In [ ]:
X_val.head()

# Making heat Map for Numerical Columns to know various correlations

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(data = X_train[Num_cols].corr(), annot = True)
plt.show()

In [ ]:
# Using MinMaxScaler
# Because it's a type of normalization (bringing the data to a scale between 0 and 1).
# Prevent Distortion Due to Outliers
min_max_scl=MinMaxScaler()
X_train=min_max_scl.fit_transform(X_train)
X_val=min_max_scl.transform(X_val)

# Repeating the same preprocessing tecniques for Test data

In [ ]:
test_data['travel_time'] = ((pd.to_datetime(test_data['tpep_pickup_datetime'])-pd.to_datetime(test_data['tpep_dropoff_datetime'])).dt.seconds)/60
test_data = test_data.drop(columns=['tpep_pickup_datetime','tpep_dropoff_datetime'])
test_data['store_and_fwd_flag'].fillna('N', inplace=True)
test_data['payment_type'].replace('unknown', 'Credit Card', inplace = True)
test_data[Num_cols]= si.transform(test_data[Num_cols])
test_data = pd.get_dummies(test_data, columns = Cat_cols, prefix=['store_and_fwd_flag ','payment_type'],dtype=bool)
test_data = test_data*1
test_data = min_max_scl.transform(test_data)

# Milestone2

# DummyRegressor

In [ ]:
dummy_regressor = DummyRegressor(strategy='mean')

In [ ]:
dummy_regressor.fit(X,y)

In [ ]:
X_test=test_data
y_pred = dummy_regressor.predict(X_test)

In [ ]:
submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(y_pred)+1)]
submission["total_amount"] = y_pred
submission.to_csv('submission.csv',index=False)

In [ ]:
submission

level2_74# MODEL 1
#  Train LinearRegression with cross validation
# Got 0.69963 score
# R2 score:0.7235502908878633

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
LR=LinearRegression()
LR.fit(X_train,y_train)


In [ ]:
y_pred=LR.predict(X_val)

In [ ]:
mse =-cross_val_score(LR,X_train,y_train,scoring='neg_mean_squared_error',cv=5)

In [ ]:
np.mean(mse)

In [ ]:
y_pred

In [ ]:
test_predictions = LR.predict(test_data)

In [ ]:
R2_score=r2_score(y_val,y_pred)
R2_score

In [ ]:
submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(test_predictions)+1)]
submission["total_amount"] = test_predictions
submission.to_csv('submission.csv',index=False)

In [ ]:
submission

# Milestone 3

# MODEL 2
# Train the model with the KNN algorithm
# Hyperperameter tuning with GridsearchCV
# R2 score:0.7893123232754766
# Got score 0.76001


In [ ]:
# Creating a pipe with standard scaler, polynomial features of degree 2 and KNN
PIPELINE = Pipeline([ ('ss', StandardScaler()), ('poly',PolynomialFeatures(degree= 2)),('pca',PCA(n_components = 8)),('KNN',KNeighborsRegressor()) ])

In [ ]:
parameters = {
    'KNN__n_neighbors': [3, 5, 7, 9],  # trying different values for n_neighbors
}

# Using GridSearchCV to find the best parameters
grid_search = GridSearchCV(PIPELINE, parameters, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)

# Fitting the GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters set:")
print(grid_search.best_params_)

# Evaluating on the validation set
y_pred = grid_search.predict(X_val)

# Calculating R^2 score or any other preferred metric
r2 = r2_score(y_val, y_pred)
print(f'R^2 Score: {r2}')

In [ ]:
test_predictions_knn = grid_search.predict(test_data)

In [ ]:
submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(test_predictions_knn)+1)]
submission["total_amount"] = test_predictions_knn
submission.to_csv('submission.csv',index=False)

In [ ]:
submission

# Milestone 4

# MODEL 3
# Train the model with DecisionTreeRegressor
# Hyperperameter tuning with GridsearchCV
# R2 Score: 0.8072189679237509
# Got score 0.8308

In [ ]:
# Creating a pipe with standard scaler, polynomial features of degree 2 and DecisionTreeRegressor
PIPELINE_dt = Pipeline([ ('ss', StandardScaler()), ('poly',PolynomialFeatures(degree= 2)),('pca',PCA(n_components = 8)),('DT',DecisionTreeRegressor(random_state=42)) ])

In [ ]:
# Define the parameter grid
param_grid = {
    'DT__max_depth': [10, 20, 30],

}

In [ ]:
# Grid Search with Cross-Validation
dt_grid_search = GridSearchCV(estimator=PIPELINE_dt, param_grid=param_grid,
                              cv=5, n_jobs=-1, scoring='neg_mean_squared_error',verbose=2)

In [ ]:
# Fit the model on the training data
dt_grid_search.fit(X_train, y_train)

In [ ]:
# Assuming X_val and y_val are your validation data and labels
y_pred_val = dt_grid_search.predict(X_val)

# Calculate R^2 score
r2_score_val = r2_score(y_val, y_pred_val)

print(f"R^2 Score on Validation Set: {r2_score_val}")


In [ ]:
predictions = dt_grid_search.predict(test_data)

In [ ]:
submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(predictions)+1)]
submission["total_amount"] =predictions
submission.to_csv('submission.csv',index=False)

In [ ]:
submission

0.895795448815323# MODEL 4
# Train the model with Random Forest regressor(Bagging Algo)
# Hyperperameter tuning with GridsearchCV
# R^2 Score: 0.895795448815323
# Got score:0.88076

In [ ]:
# Define the pipeline
pipe_rfr = Pipeline([
    ('ss', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('pca', PCA(n_components=8)),
    ('rfr', RandomForestRegressor(random_state=42))
])

In [ ]:
param_grid_rfr = {
    'rfr__n_estimators': [100, 200, 300],
}

In [ ]:
# Set up the GridSearchCV with a default parameter grid
grid_search_rfr = GridSearchCV(estimator=pipe_rfr, param_grid=param_grid_rfr, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)

# Fit the GridSearchCV to the training data
grid_search_rfr.fit(X_train, y_train)

In [ ]:
# Predict on the validation set
y_pred_val = grid_search_rfr.predict(X_val)

# Calculate R^2 score
r2 = r2_score(y_val, y_pred_val)
print(f"R^2 Score: {r2}")

In [ ]:
prediction_rfr = grid_search_rfr.predict(test_data)

In [ ]:
submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(prediction_rfr)+1)]
submission["total_amount"] =prediction_rfr
submission.to_csv('submission.csv',index=False)

In [ ]:
submission

# MODEL 5
# ExtraTreesRegressor
# Hyperperameter tuning with GridsearchCV
# R^2 Score: 0.9134102150234041
# Got score:0.89369

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
pipe_etr = Pipeline([
    ('ss', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('pca', PCA(n_components=8)),
    ('etr',ExtraTreesRegressor(random_state=42))
])

In [ ]:
param_grid = {
    'etr__n_estimators': [100, 200, 300]
}

In [ ]:
# GridSearchCV
grid_search = GridSearchCV(estimator=pipe_etr, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2', verbose=2)
grid_search.fit(X_train, y_train)

In [ ]:
# Best model
best_model = grid_search.best_estimator_

# Predict on validation set
y_pred = best_model.predict(X_val)

# R^2 Score
r2 = r2_score(y_val, y_pred)
print(f'R^2 Score: {r2}')

In [ ]:
# Generate predictions for the test set
predictions_etr= best_model.predict(test_data)

In [ ]:
submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(predictions_etr)+1)]
submission["total_amount"] =predictions_etr
submission.to_csv('submission.csv',index=False)

In [ ]:
submission

# MODEL 6
# Train the model with XGBoost (Extreme Gradient Boosting)
# Hyperperameter tuning with GridsearchCV
# R^2 Score: 0.8986797939172592
# Got score: 0.87837

In [ ]:
import xgboost as xgb

In [ ]:
# Creating a pipe with standard scaler, polynomial features of degree 2 and DecisionTreeRegressor
PIPELINE_XGB = Pipeline([ ('ss', StandardScaler()), ('poly',PolynomialFeatures(degree= 2)),('pca',PCA(n_components = 8)),('XGB', xgb.XGBRegressor(objective='reg:squarederror', random_state=42)) ])

In [ ]:
param_grid = {
    'XGB__n_estimators': [100, 200, 300]
}

In [ ]:
# GridSearchCV
grid_search = GridSearchCV(estimator=PIPELINE_XGB, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)


In [ ]:
# Best estimator
best_estimator = grid_search.best_estimator_

# Making predictions on the validation set
y_pred_val = best_estimator.predict(X_val)

In [ ]:
# Calculating R2 score
r2 = r2_score(y_val, y_pred_val)
print(f"R2 score on the validation set: {r2}")

In [ ]:
# Make predictions on the test data
predictions_xgb = best_estimator.predict(test_data)

In [ ]:
submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(predictions_xgb)+1)]
submission["total_amount"] =predictions_xgb
submission.to_csv('submission.csv',index=False)

In [ ]:
submission

# MODEL 7
# MLP Model(Multilayer perceptron model)
# Hyperperameter tuning with GridsearchCV
# R^2 Score: 0.943500910034381
# Got score:0.91468



In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
pipe_mlp = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPRegressor(random_state=42))
])

In [ ]:
param_grid = {
    'mlp__activation': ['relu', 'tanh'],
}

In [ ]:
grid_search = GridSearchCV(estimator=pipe_mlp, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

In [ ]:
# Train the model
grid_search.fit(X_train, y_train)

# Print best parameters
print("Best parameters found: ", grid_search.best_params_)


In [ ]:
# Predictions on test set
y_pred = grid_search.predict(X_val)

# Calculate R2 score
r2 = r2_score(y_val, y_pred)
print("R2 score: ", r2)


In [ ]:
predictions_mlp = grid_search.predict(test_data)

In [ ]:
submission = pd.DataFrame(columns = ["ID","total_amount"])
submission["ID"] = [i for i in range(1,len(predictions_mlp)+1)]
submission["total_amount"] =predictions_mlp
submission.to_csv('submission.csv',index=False)

In [ ]:
submission

# **MLP Model(Multilayer perceptron model)showed the best score which is 0.91468**
# **R^2 Score: 0.943500910034381**